### Dataset Content Description
- **timestamp** is the moment the question was given, represented as Unix timestamp in milliseconds.
- **solving_id** represents each learning session of students corresponds to each bunle. It is a form of single
integer, starting from 1 .
- **question_id** is the ID of the question that given to student, which is a form of q{integer}.
- **user_answer** is the answer that the student submitted, recorded as a character between a and d inclusively.
- **elapsed_time** is the time that the students spends on each question in milliseconds.


### Our Dataset
We plan to reduce the data to have only columns for student_id, question_id, bundle_id, tags, elapsed_time, and correct
- **student_id** is the ID of the student. We will not use this for training, but it's necessary for sorting so that we don't leak data
- **bundle_id** is the bundle that a question is in. Bundles are sets of questions that reference the same passage/image/video/etc
- **tags** are the skills associated with a certain question. Most question have multiple tags.
- **Correct** is whether or not the student got the answer correct in that interaction

In [2]:
import os
import polars as pl # Using polars instead of pandas for speed. >9 million lines in 784k csv files.
from tqdm import tqdm
import pandas as pd
import pyarrow as pa

#### Data Merging Preparation
Our original dataset is comprised of one csv file per each of the 784k students, which results in massive overhead when reading in data. To prevent this, we're taking all of the relevant data and merging it into one unified dataset that can be easily read, navigated, and edited.

In [ ]:
kt1_dir = "Data/KT1"
questions_fname = "Data/contents/questions.csv"

# Load important columns of questions file (finding correct_answer and tags)
# This only needs to be done once but we'll reference it multiple times per student interaction
questions = (
    pl.read_csv(questions_fname)
    .with_columns([
        pl.col("question_id").str.replace("q", "").cast(pl.Int32),
        pl.col("bundle_id").str.replace("b", "").cast(pl.Int32),
        pl.col("tags").cast(pl.Utf8)
    ])
    .select(["question_id", "correct_answer", "bundle_id", "tags"])
)


student_files = [os.path.join(kt1_dir, f) for f in os.listdir(kt1_dir) if f.endswith(".csv")]

dfs = []

### Data Fetching and Merging
Here, we take all of the information that we need from each KT1 file and combine it into a single .csv

In [ ]:
# For each interaction, we take the student_id, question_id, bundle_id, tags, elapsed_time, and whether they answered correctly
for file in tqdm(student_files, desc="Progress"):
    # Take student_id from filename, remove 'u' prefix to make it int
    student_id = int(os.path.basename(file).replace("u", "").replace(".csv", ""))

    df = (
        pl.read_csv(file)
        .with_columns([
            pl.lit(student_id).alias("student_id"),
            pl.col("question_id").str.replace("q", "").cast(pl.Int32), # Remove 'q' prefix to make question_id int
        ])
        .join(questions, on="question_id", how="left")
        .with_columns([
            # Adds 'correct' column, which details if student got the question correct
            (pl.col("user_answer").str.strip_chars().str.to_lowercase() == pl.col("correct_answer").str.strip_chars().str.to_lowercase())
            .cast(pl.Int8)
            .alias("correct")
        ])
        .select(["student_id", "question_id", "bundle_id", "tags", "elapsed_time", "correct"])
    )

    # Tags are currently in a list, we need to flatten them so they work in csv
    df = df.with_columns(
        pl.col("tags")
        .cast(pl.Utf8)
        .str.replace_all(r"\[|\]|\s", "")
        .str.replace_all(",", ";")
        .alias("tags")
    )

    dfs.append(df)

# Sort by student_id, then question_id
final_df = pl.concat(dfs, how="vertical").sort(["student_id", "question_id"])

fname = "combined_dataset.csv"
final_df.write_csv(fname)
print(f"Saved {fname}")

### Model 1 Baseline Accuracy

In [9]:
data = pl.read_csv(rf".\Data\combined_dataset.csv")
df = data.to_pandas() # Convert from polars to pandas

correct_count = df['correct'].value_counts()
student_acc = correct_count.get(1) / (correct_count.get(0) + correct_count.get(1))
print(f"Overall student accuracy: {student_acc:.4%}")

Overall student accuracy: 65.3418%


Our baseline accuracy to compare against is 65.3418%, which is what we would get if the model guessed that the student will be 'correct' every time.